In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
from collections import Counter

In [3]:
# df 모두 total_code_1007(total_code_1007 빈백 처리(실행본)) 돌리고 저장한 csv
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seojin/data/train_df_v2.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/seojin/data/test_df_v2.csv')

In [4]:
len(test_df)

23405

In [5]:
train_df.head()

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최",용인문화재단 인문학 콘서트 뮤지컬 이야기 개최 인문학 콘서트 뮤지컬 이야기 용인문화...
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원",농촌 테마파크 단체 체험객 체험료 체험일 기준 용인시통합예약사이트 접수 용인특례시 ...
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장",노후 주택 에너지 성능 개선 신청 연장 용인시청 노후 건축물 환경친화적 개선 온실 ...
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대",수원 고양시 특례시 지정 도시경쟁력 증가 도내 인구 수원 고양 특례시 지정 골자 지...
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할",스페인 미국 국제 명예 자문관 위촉 역할 대외 홍보 권태면 주코스타리카 대사 이정관...


In [6]:
test_df.head()

,ID,제목,키워드
0,TEST_00000,[부고] 김태수씨 별세 외,김태수 별세 김태수씨 광남초등학 교장 별세 김윤정 이노코리아 희정 한성대 교수 도식...
1,TEST_00001,"신규 확진 나흘째 세자릿수... 방역당국, 핼러윈 풍선효과 차단 총력",신규 확진 나흘 세자릿수 방역당국 핼러윈 풍선 효과 차단 총력 감염증 신종 코로나바...
2,TEST_00002,"[서경이 만난 사람] 전해철 장관 ""재정분권 강화 '지방자치 2.0 시대' 마중물 ...",전해철 장관 재정 분권 강화 지방자치 시대 마중물 장관 전해철 행정안전부 풀뿌리 민...
3,TEST_00003,"용인시, 12일 '장애인 구인 구직 만남의 날' 채용 행사",구인 장애인 구직 만남 채용 행사 노호근 용인특례시 장애인 취업 대회의실 일자리 센...
4,TEST_00004,지자체 벽 터 경기지역 산단 활성화 모색,지자체 북동부 산업단지 혁신단위 설정 전략 연계 특성 제시 경기도경제과학진흥원 보고...


In [7]:
train_df = train_df.drop(columns=['ID', '제목'])
test_df = test_df.drop(columns=['ID', '제목'])

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

In [9]:
torch.cuda.empty_cache()

In [10]:
config = {
    "learning_rate": 2e-5,
    "epoch": 5,
    "batch_size": 32,
    "weight_decay": 0.01 #과적합 방지를 위한 가중치 감쇠
}


In [11]:
CFG = SimpleNamespace(**config)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
# # 토크나이저와 모델 변경
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertTokenizer, RobertaForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=len(train_df['분류'].unique())).to(device)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

# model = RobertaForSequenceClassification.from_pretrained('klue/roberta-large', num_labels=len(train_df['분류'].unique())).to(device)
# tokenizer = RobertaTokenizer.from_pretrained('klue/roberta-large')

# tokenizer = BertTokenizer.from_pretrained('klue/roberta-large')
# model = RobertaForSequenceClassification.from_pretrained('klue/roberta-large',num_labels=len(train_df['분류'].unique())).to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [14]:
# 데이터 준비
#train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
#test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [18]:
# train_dataset[0]

In [15]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate, weight_decay=CFG.weight_decay)
# optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
# 학습
model.train()
for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

Validating: 100%|██████████| 340/340 [01:59<00:00,  2.84it/s]


Validation F1 Score: 0.49


Validating: 100%|██████████| 340/340 [01:59<00:00,  2.84it/s]


Validation F1 Score: 0.58


Validating: 100%|██████████| 340/340 [01:59<00:00,  2.85it/s]


Validation F1 Score: 0.60


Validating: 100%|██████████| 340/340 [01:59<00:00,  2.85it/s]


Validation F1 Score: 0.64


Validating: 100%|██████████| 340/340 [01:59<00:00,  2.84it/s]

Validation F1 Score: 0.64


In [17]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 732/732 [04:17<00:00,  2.84it/s]


In [ ]:
# model

In [18]:
len(test_df)

23405

In [19]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seojin/data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions

sample_submission.to_csv("/content/drive/MyDrive/Colab Notebooks/seojin/baseline_kobert_1008.csv", encoding='UTF-8-sig', index=False)

In [3]:
result = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/seojin/baseline_kobert_1008.csv")
result.head()

,ID,분류
0,TEST_00000,사회:사회일반
1,TEST_00001,사회:사회일반
2,TEST_00002,정치:행정_자치
3,TEST_00003,경제:취업_창업
4,TEST_00004,지역


In [4]:
result['분류'].value_counts()

,count
분류,
지역,10631
경제:부동산,1622
사회:사건_사고,1201
경제:반도체,1118
사회:사회일반,611
사회:의료_건강,502
사회:교육_시험,500
정치:국회_정당,455
정치:선거,388


In [5]:
len(result)

23405